# Empezamos a generar las funciones de la API

In [1]:
import pandas as pd
from fastapi import FastAPI, HTTPException


http://127.0.0.1:8000

Buscamos crear la primer función solicitada!

Creamos la API

In [11]:
app = FastAPI()

In [4]:
@app.get("/")
def read_root():
    return {"mensaje": "¡Bienvenido a mi API!"}

-------
-----
-----

# FUNCIÓN 1

---
---
---

## Función: PlayTimeGenre

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

Data frames: users_items y steam_games
- Horas jugadas esta en: users_items['playtime_forever']
- Genero del juago: steam_games['genres']
- Año lanzamiento: steam_games['year']

Dato de ingreso: genero <br>
Devuelve: Año y horas jugadas

In [55]:
# Cargo los dataframe a usar

#parquet_ruta1 = "data/users_items_base1.parquet"
csv_ruta_1 = "data/users_items_base1.csv"
csv_ruta_2 = "data/steam_games_base1.csv"
csv_ruta_3 = "data/users_reviews_base1.csv"

#users_items = pd.read_parquet(parquet_ruta1)
users_items = pd.read_csv(csv_ruta_1)
steam_games = pd.read_csv(csv_ruta_2)
users_reviews = pd.read_csv(csv_ruta_3)

Para la primer funcion solo quiero las columnas año, playtime_forever, id, steam_id, genres

In [56]:
columnas_sg = ['id', 'genres', 'title', 'year']
steam_games_fc1 = steam_games[columnas_sg]

In [57]:
steam_games_fc1

,id,genres,title,year
0,761140,Action,Lost Summoner Kitty,2018
1,761140,Casual,Lost Summoner Kitty,2018
2,761140,Indie,Lost Summoner Kitty,2018
3,761140,Simulation,Lost Summoner Kitty,2018
4,761140,Strategy,Lost Summoner Kitty,2018
...,...,...,...,...
34270,745400,Adventure,Kebab it Up!,2018
34271,745400,Casual,Kebab it Up!,2018
34272,745400,Indie,Kebab it Up!,2018
34273,658870,Casual,EXIT 2 - Directions,2017


In [58]:
users_items.columns

Index(['Unnamed: 0', 'user_id', 'items_count', 'steam_id', 'item_id',
       'item_name', 'playtime_forever', 'playtime_2weeks'],
      dtype='object')

In [59]:
columnas_ui = ['steam_id', 'item_id','item_name', 'playtime_forever']
users_items_fc1 = users_items[columnas_ui]

users_items_fc1

,steam_id,item_id,item_name,playtime_forever
0,76561197970982479,10.0,Counter-Strike,6.0
1,76561197970982479,20.0,Team Fortress Classic,0.0
2,76561197970982479,30.0,Day of Defeat,7.0
3,76561197970982479,40.0,Deathmatch Classic,0.0
4,76561197970982479,50.0,Half-Life: Opposing Force,0.0
...,...,...,...,...
5113966,76561198329548331,373330.0,All Is Dust,0.0
5113967,76561198329548331,388490.0,One Way To Die: Steam Edition,3.0
5113968,76561198329548331,521570.0,You Have 10 Seconds 2,4.0
5113969,76561198329548331,519140.0,Minds Eyes,3.0


In [60]:
df_funcion_PlaytimeGenre = pd.merge(users_items_fc1, steam_games_fc1, left_on='item_name', right_on='title', how='inner')

df_funcion_PlaytimeGenre

,steam_id,item_id,item_name,playtime_forever,id,genres,title,year
0,76561197970982479,10.0,Counter-Strike,6.0,10,Action,Counter-Strike,2000
1,76561198035864385,10.0,Counter-Strike,0.0,10,Action,Counter-Strike,2000
2,76561197963445855,10.0,Counter-Strike,0.0,10,Action,Counter-Strike,2000
3,76561198002099482,10.0,Counter-Strike,93.0,10,Action,Counter-Strike,2000
4,76561198007659921,10.0,Counter-Strike,108.0,10,Action,Counter-Strike,2000
...,...,...,...,...,...,...,...,...
7410594,76561198071808318,444770.0,Mimpi Dreams,46.0,444770,Indie,Mimpi Dreams,2016
7410595,76561198107283457,354280.0,ChaosTower,164.0,354280,Action,ChaosTower,2016
7410596,76561198107283457,354280.0,ChaosTower,164.0,354280,Casual,ChaosTower,2016
7410597,76561198107283457,354280.0,ChaosTower,164.0,354280,Indie,ChaosTower,2016


# Play Time Genre

df_funcion_PlaytimeGenre

In [61]:
@app.get('/PlayTimeGenre/{genero}')
def PlayTimeGenre(genero):
    

    # Filtrar el DataFrame por el género especificado
    df_genero = df_funcion_PlaytimeGenre[df_funcion_PlaytimeGenre['genres'] == genero]

    # Agrupar por año y calcular la suma de las horas jugadas
    resumen_por_anio = df_genero.groupby('year')['playtime_forever'].sum()

    # Encontrar el año con más horas jugadas
    anio_con_mas_horas = resumen_por_anio.idxmax()

    # Encontrar la cantidad total de horas jugadas en ese año
    horas_totales = resumen_por_anio.max()

    return f"""
Género: {genero}
Año con más horas jugadas para el género: {anio_con_mas_horas}.
Horas totales jugadas: {horas_totales}
"""

In [62]:
# Guardo en df_funcion_PlaytimeGenre en un .csv
# df_funcion_PlaytimeGenre.to_csv("df_funciones/df_funcion_PlayTimeGenre.csv", index=False)

# Comentado, por que se hace una sola vez!

In [63]:
# Ejemplo de uso
genero_deseado = 'Indie'
resultado = PlayTimeGenre(genero_deseado)
print(resultado)


Género: Indie
Año con más horas jugadas para el género: 2006.
Horas totales jugadas: 444992091.0



In [64]:
genero = 'Action'
resultado2 = PlayTimeGenre(genero)
print(resultado2)


Género: Action
Año con más horas jugadas para el género: 2012.
Horas totales jugadas: 1009371451.0



Ya tengo la primer función! <br>
Tengo que ver la parte tecnica para ver como toma ese DataFrame!

-------
-----
-----

# FUNCIÓN 2

---
---
---


## Función: User For Genre

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

datos necesarios:
- hs jugadas
- ususario
- genero

input: Genero <br>
output: Ususario, max hs jugadas

In [65]:
steam_games.head()

,Unnamed: 0,publisher,genres,app_name,title,release_date,price,id,developer,year
0,0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
1,1,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
2,2,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
3,3,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
4,4,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018


In [66]:
users_items.head()

,Unnamed: 0,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,10.0,Counter-Strike,6.0,0.0
1,1,76561197970982479,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0
2,2,76561197970982479,277,76561197970982479,30.0,Day of Defeat,7.0,0.0
3,3,76561197970982479,277,76561197970982479,40.0,Deathmatch Classic,0.0,0.0
4,4,76561197970982479,277,76561197970982479,50.0,Half-Life: Opposing Force,0.0,0.0


In [67]:
# Selecciono las columnas necesarias para cada DataFrame
col_sg_2 = ['genres', 'title']
col_ui_2 = ['user_id','playtime_forever','item_name']

# Filtro con las columnas seleccionadas
steam_games_fc2 = steam_games[col_sg_2]
users_items_fc2 = users_items[col_ui_2]

# Genero la union de ambos DataFrame
df_funcion_UserForGenre = pd.merge(users_items_fc2, steam_games_fc2, left_on='item_name', right_on='title', how='inner')

df_funcion_UserForGenre


,user_id,playtime_forever,item_name,genres,title
0,76561197970982479,6.0,Counter-Strike,Action,Counter-Strike
1,js41637,0.0,Counter-Strike,Action,Counter-Strike
2,Riot-Punch,0.0,Counter-Strike,Action,Counter-Strike
3,doctr,93.0,Counter-Strike,Action,Counter-Strike
4,corrupted_soul,108.0,Counter-Strike,Action,Counter-Strike
...,...,...,...,...,...
7410594,76561198071808318,46.0,Mimpi Dreams,Indie,Mimpi Dreams
7410595,76561198107283457,164.0,ChaosTower,Action,ChaosTower
7410596,76561198107283457,164.0,ChaosTower,Casual,ChaosTower
7410597,76561198107283457,164.0,ChaosTower,Indie,ChaosTower


# User For Genre

df_funcion_UserForGame

In [68]:
@app.get('/UserForGenre/{genero}')
def UserForGenre(genero):
    
    # Filtrar el DataFrame por el género especificado
    df_genero = df_funcion_UserForGenre[df_funcion_UserForGenre['genres'] == genero]

    # Agrupar por usuario y calcular la suma de las horas jugadas
    resumen_por_user = df_genero.groupby('user_id')['playtime_forever'].sum()

    # Encontrar el usuario con más horas jugadas
    user_con_mas_horas = resumen_por_user.idxmax()

    # Encontrar la cantidad total de horas jugadas por el usuario
    horas_totales = resumen_por_user.max()

    return f"""
Género: {genero}
Usuario con más horas jugadas para el género: {user_con_mas_horas}.
Horas totales jugadas: {horas_totales}
"""

In [69]:
# Guardo en df_funcion_PlaytimeGenre en un .csv
# df_funcion_UserForGenre.to_csv("df_funciones/df_user_for_genre.csv", index=False)

# Comentado, por que se hace una sola vez!

In [70]:
# Prueba 1
genero2 = 'Action'

fc2_resultado = UserForGenre(genero2)
print(fc2_resultado)


Género: Action
Usuario con más horas jugadas para el género: shinomegami.
Horas totales jugadas: 1322018.0



In [71]:
# Prueba 2
genero3 = 'Simulation'

fc2_resultado2 = UserForGenre(genero3)
print(fc2_resultado2)


Género: Simulation
Usuario con más horas jugadas para el género: Evilutional.
Horas totales jugadas: 664023.0



-------
-----
-----

# FUNCIÓN 3

---
---
---

# Función: Users Recommend

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

input: año <br>
return: top 3 juegos recomendados

In [72]:
steam_games.head()

,Unnamed: 0,publisher,genres,app_name,title,release_date,price,id,developer,year
0,0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
1,1,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
2,2,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
3,3,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
4,4,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018


In [73]:
users_reviews.head()

,Unnamed: 0,user_id,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,0,76561197970982479,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,2
1,1,76561197970982479,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,2
2,2,76561197970982479,NaN,"Posted April 21, 2011.",NaN,43110.0,No ratings yet,True,2
3,3,js41637,NaN,"Posted June 24, 2014.",NaN,251610.0,15 of 20 people (75%) found this review helpful,True,2
4,4,js41637,NaN,"Posted September 8, 2013.",NaN,227300.0,0 of 1 people (0%) found this review helpful,True,2


In [74]:
users_reviews.columns

Index(['Unnamed: 0', 'user_id', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'sentiment_analysis'],
      dtype='object')

In [75]:
users_items.head()

,Unnamed: 0,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,10.0,Counter-Strike,6.0,0.0
1,1,76561197970982479,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0
2,2,76561197970982479,277,76561197970982479,30.0,Day of Defeat,7.0,0.0
3,3,76561197970982479,277,76561197970982479,40.0,Deathmatch Classic,0.0,0.0
4,4,76561197970982479,277,76561197970982479,50.0,Half-Life: Opposing Force,0.0,0.0


In [76]:
users_reviews_fc3 = users_reviews.copy()

In [77]:
# Extraer el año usando expresiones regulares y crear una nueva columna 'year'
users_reviews_fc3['year'] = users_reviews_fc3['posted'].str.extract(r'(\d{4})')

# Convertir la columna 'year' a tipo numérico (si es necesario)
users_reviews_fc3['year'] = pd.to_numeric(users_reviews_fc3['year'], errors='coerce')

users_reviews_fc3.head(2)


,Unnamed: 0,user_id,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis,year
0,0,76561197970982479,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,2,2011.0
1,1,76561197970982479,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,2,2011.0


In [78]:
# Eliminar filas con NaN en la columna 'year'
users_reviews_fc3 = users_reviews_fc3.dropna(subset=['year'])

users_reviews_fc3.head(2)


,Unnamed: 0,user_id,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis,year
0,0,76561197970982479,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,2,2011.0
1,1,76561197970982479,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,2,2011.0


In [79]:
# Verificar si hay NaN en la columna 'year'
nan_values = users_reviews_fc3['year'].isna().sum()

# Mostrar el resultado
print("Número de NaN en la columna 'year':", nan_values)

Número de NaN en la columna 'year': 0


In [80]:
# Convertir la columna 'year' a tipo entero
users_reviews_fc3['year'] = users_reviews_fc3['year'].astype('int')

users_reviews_fc3.head(2)


,Unnamed: 0,user_id,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis,year
0,0,76561197970982479,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,2,2011
1,1,76561197970982479,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,2,2011


In [81]:
año_maximo = users_reviews_fc3['year'].max()
año_maximo

2015

In [82]:
año_min = users_reviews_fc3['year'].min()
año_min

2010

In [83]:

# Seleccion de columnas
col_ur_3 = ['recommend', 'sentiment_analysis', 'item_id','year']
col_ui_3 = ['item_name', 'item_id']

# Filtro con las columnas seleccionadas
users_items_fc3 = users_items[col_ui_3]
users_reviews_fc3 = users_reviews_fc3[col_ur_3]



In [84]:
# Eliminar filas con NaN
users_items_fc3 = users_items_fc3.dropna()

users_items_fc3

,item_name,item_id
0,Counter-Strike,10.0
1,Team Fortress Classic,20.0
2,Day of Defeat,30.0
3,Deathmatch Classic,40.0
4,Half-Life: Opposing Force,50.0
...,...,...
5113965,BrainBread 2,346330.0
5113966,All Is Dust,373330.0
5113967,One Way To Die: Steam Edition,388490.0
5113968,You Have 10 Seconds 2,521570.0


In [85]:
# Cuento cuántos item_id diferentes hay
cantidad_item_id_diferentes = users_items_fc3['item_id'].nunique()

# Imprimo el resultado
print("Cantidad de item_id diferentes:", cantidad_item_id_diferentes)

Cantidad de item_id diferentes: 10978


In [86]:
# Filtrar el DataFrame para obtener un listado de juegos sin repetidos
listado_juegos_sin_repetidos = users_items_fc3.drop_duplicates(subset='item_id')

# Controlo la cantidad de filas devueltas con el resultado anterior! --> OK!
listado_juegos_sin_repetidos

,item_name,item_id
0,Counter-Strike,10.0
1,Team Fortress Classic,20.0
2,Day of Defeat,30.0
3,Deathmatch Classic,40.0
4,Half-Life: Opposing Force,50.0
...,...,...
4870097,NOBUNAGA'S AMBITION: Souzou SengokuRisshiden,375450.0
5052835,Alienware Steam Machine,353390.0
5065227,ChaosTower,354280.0
5079812,Aveyond 4: Shadow Of The Mist,433920.0


In [87]:
# Eliminar filas con NaN
users_reviews_fc3 = users_reviews_fc3.dropna()

users_reviews_fc3

,recommend,sentiment_analysis,item_id,year
0,True,2,1250.0,2011
1,True,2,22200.0,2011
2,True,2,43110.0,2011
3,True,2,251610.0,2014
4,True,2,227300.0,2013
...,...,...,...,...
58406,True,1,730.0,2015
58409,True,2,253980.0,2015
58419,True,0,730.0,2015
58421,True,2,730.0,2015


In [88]:
año_min = users_reviews_fc3['year'].min()
año_min

2010

In [89]:
año_maximo = users_reviews_fc3['year'].max()
año_maximo

2015

## Users Recommend

users_reviews_fc3 <br>
listado_juegos_sin_repetidos

In [90]:
@app.get('/UsersRecommend/{anio}')
def UsersRecommend(año: int):
    # Calcula el margen de años donde hay datos
    año_maximo = users_reviews_fc3['year'].max()
    año_min = users_reviews_fc3['year'].min()

    if año < año_min or año > año_maximo:
        
        print('No hay datos para el año indicado!')
        print('Se debe colocar un año entre', año_min, 'y', año_maximo)
    
    else:
        # Filtro el año dado con recomendación igual a 'True' y sentimiento positivo/neutro
        filtro_reviews = users_reviews_fc3[(users_reviews_fc3['year'] == año) &
                                        (users_reviews_fc3['recommend'] == True) &
                                        (users_reviews_fc3['sentiment_analysis'] >= 1)]

        # Agrupo por item_id y suma el sentiment_analysis
        sentiment_sum = filtro_reviews.groupby('item_id')['sentiment_analysis'].sum().reset_index(name='sentiment_sum')

        # Ordeno de forma descendente según la suma de 'sentiment_sum'
        ordenados_sentiments = sentiment_sum.sort_values(by='sentiment_sum', ascending=False)

        # Asocia los item_id con los nombres de los juegos
        df_merge = pd.merge(ordenados_sentiments, listado_juegos_sin_repetidos, on='item_id', how='left')

        # Elimino las filas con NaN
        df_merge = df_merge.dropna()

        # Filtro el top 3
        top_3_con_nombres = df_merge.head(3)

        # Crea la lista de resultados en el formato deseado con nombres de juegos
        result = [{"Puesto {}: ".format(i+1): item_name} for i, item_name in enumerate(top_3_con_nombres['item_name'])]

        

        return result

In [91]:
# Ejemplo de uso
año_deseado = 2014
resultado = UsersRecommend(año_deseado)
resultado

[{'Puesto 1: ': 'Counter-Strike: Global Offensive'},
 {'Puesto 2: ': "Garry's Mod"},
 {'Puesto 3: ': 'Unturned'}]

Ojo, si se carga con parquet va a fallar por cambio de tipo de datos!

In [92]:
# Guardo los DataFrame utilizados en un .csv
# users_reviews_fc3.to_csv("df_funciones/users_reviews_fc3.csv", index=False)
# listado_juegos_sin_repetidos.to_csv("df_funciones/listado_juegos_sin_repetidos.csv", index=False)



# Comentado, por que se hace una sola vez!

-------
-----
-----

# FUNCIÓN 4

---
---
---

## Función: User Worst Developer

def UsersWorstDeveloper( año : int ): Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [93]:
users_reviews_fc4 = users_reviews.copy()

In [94]:
steam_games_fc4 = steam_games.copy()

In [95]:
users_items_fc4 = users_items.copy()

In [96]:
users_items_fc4.head(2)

,Unnamed: 0,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,10.0,Counter-Strike,6.0,0.0
1,1,76561197970982479,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0


In [97]:
users_reviews_fc4.head(2)

,Unnamed: 0,user_id,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,0,76561197970982479,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,2
1,1,76561197970982479,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,2


In [98]:
steam_games_fc4.head(2)

,Unnamed: 0,publisher,genres,app_name,title,release_date,price,id,developer,year
0,0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
1,1,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018


In [99]:
# Extraer el año usando expresiones regulares y crear una nueva columna 'year'
users_reviews_fc4['year'] = users_reviews_fc4['posted'].str.extract(r'(\d{4})')

# Convertir la columna 'year' a tipo numérico (si es necesario)
users_reviews_fc4['year'] = pd.to_numeric(users_reviews_fc4['year'], errors='coerce')


users_reviews_fc4

,Unnamed: 0,user_id,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis,year
0,0,76561197970982479,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,2,2011.0
1,1,76561197970982479,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,2,2011.0
2,2,76561197970982479,NaN,"Posted April 21, 2011.",NaN,43110.0,No ratings yet,True,2,2011.0
3,3,js41637,NaN,"Posted June 24, 2014.",NaN,251610.0,15 of 20 people (75%) found this review helpful,True,2,2014.0
4,4,js41637,NaN,"Posted September 8, 2013.",NaN,227300.0,0 of 1 people (0%) found this review helpful,True,2,2013.0
...,...,...,...,...,...,...,...,...,...,...
58454,58454,76561198312638244,NaN,Posted July 10.,NaN,70.0,No ratings yet,True,2,NaN
58455,58455,76561198312638244,NaN,Posted July 8.,NaN,362890.0,No ratings yet,True,2,NaN
58456,58456,LydiaMorley,1 person found this review funny,Posted July 3.,NaN,273110.0,1 of 2 people (50%) found this review helpful,True,2,NaN
58457,58457,LydiaMorley,NaN,Posted July 20.,NaN,730.0,No ratings yet,True,2,NaN


In [100]:
# Eliminar filas con NaN en la columna 'year'
users_reviews_fc4 = users_reviews_fc4.dropna(subset=['year'])

users_reviews_fc4.head(2)

,Unnamed: 0,user_id,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis,year
0,0,76561197970982479,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,2,2011.0
1,1,76561197970982479,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,2,2011.0


In [101]:
# Convertir la columna 'year' a tipo entero
users_reviews_fc4['year'] = users_reviews_fc4['year'].astype('int')

users_reviews_fc4.head(2)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_15120\405261573.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_reviews_fc4['year'] = users_reviews_fc4['year'].astype('int')


,Unnamed: 0,user_id,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis,year
0,0,76561197970982479,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,2,2011
1,1,76561197970982479,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,2,2011


In [102]:
steam_games_fc4.head(2)

,Unnamed: 0,publisher,genres,app_name,title,release_date,price,id,developer,year
0,0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
1,1,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018


In [103]:
users_items_fc4.head(2)

,Unnamed: 0,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,10.0,Counter-Strike,6.0,0.0
1,1,76561197970982479,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0


Forma de conectar las tablas:<br>

users_reviews['item_id'] -- users_items['item_id'] <br>
steam_games['title] -- users_items['item_name'] <br>

In [126]:
# Seleccion de columnas
col_ur_4 = ['item_id','recommend', 'sentiment_analysis', 'year']
col_ui_4 = ['item_id','item_name'] 
col_sg_4 = ['developer','title']

# Filtro con las columnas seleccionadas
users_items_fc4 = users_items_fc4[col_ui_4]
users_reviews_fc4 = users_reviews_fc4[col_ur_4]
steam_games_fc4 = steam_games_fc4[col_sg_4]

In [105]:
# Eliminar filas con NaN
steam_games_fc4 = steam_games_fc4.dropna()

# Filtrar el DataFrame para obtener un listado de juegos sin repetidos
steam_games_fc4 = steam_games_fc4.drop_duplicates()

# Queda un listado unico de juegos con desarrolladores
steam_games_fc4

,developer,title
0,Kotoshiro,Lost Summoner Kitty
5,Secret Level SRL,Ironbound
9,Poolians.com,Real Pool 3D - Poolians
14,彼岸领域,弹炸人2222
17,Trickjump Games Ltd,Battle Royale Trainer
...,...,...
34264,Valve,Counter-Strike: Condition Zero
34265,Phil Fortier,Snail Trek - Chapter 3: Lettuce Be
34267,INGAME,Bravium
34269,Bidoniera Games,Kebab it Up!


In [106]:
# Eliminar filas con NaN
users_items_fc4 = users_items_fc4.dropna()

# Filtrar el DataFrame para obtener un listado de juegos sin repetidos
users_items_fc4 = users_items_fc4.drop_duplicates()

# Queda un listado unico de juegos con sis id
users_items_fc4

,item_id,item_name
0,10.0,Counter-Strike
1,20.0,Team Fortress Classic
2,30.0,Day of Defeat
3,40.0,Deathmatch Classic
4,50.0,Half-Life: Opposing Force
...,...,...
4870097,375450.0,NOBUNAGA'S AMBITION: Souzou SengokuRisshiden
5052835,353390.0,Alienware Steam Machine
5065227,354280.0,ChaosTower
5079812,433920.0,Aveyond 4: Shadow Of The Mist


In [107]:
# Asocia los item_id con los nombres de los juegos
lista_de_juegos_fc4 = pd.merge(users_items_fc4, steam_games_fc4, left_on='item_name', right_on='title', how='inner')
lista_de_juegos_fc4 = lista_de_juegos_fc4.drop(columns='item_name')

In [108]:
lista_de_juegos_fc4.columns

Index(['item_id', 'developer', 'title'], dtype='object')

In [109]:
users_reviews_fc4.columns

Index(['item_id', 'recommend', 'sentiment_analysis', 'year'], dtype='object')

In [110]:
# Unir los DataFrames usando 'item_id', de esta forma agrego la columna 'developer' y dejo un solo dataframe
df_funcion_UsersWorstDeveloper = pd.merge(users_reviews_fc4, lista_de_juegos_fc4[['item_id', 'developer', 'title']], on='item_id', how='left')

df_funcion_UsersWorstDeveloper


,item_id,recommend,sentiment_analysis,year,developer,title
0,1250.0,True,2,2011,Tripwire Interactive,Killing Floor
1,22200.0,True,2,2011,ACE Team,Zeno Clash
2,43110.0,True,2,2011,NaN,NaN
3,251610.0,True,2,2014,NaN,NaN
4,227300.0,True,2,2013,SCS Software,Euro Truck Simulator 2
...,...,...,...,...,...,...
48493,730.0,True,1,2015,Valve,Counter-Strike: Global Offensive
48494,253980.0,True,2,2015,Starbreeze,Enclave
48495,730.0,True,0,2015,Valve,Counter-Strike: Global Offensive
48496,730.0,True,2,2015,Valve,Counter-Strike: Global Offensive


## Users Worst Developer

Uso el DataFrame df_funcion_UsersWorstDeveloper

In [111]:
@app.get('/UsersWorstDeveloper/{anio}')
def UsersWorstDeveloper(año: int):
    # Calcula el margen de años donde hay datos
    año_maximo = df_funcion_UsersWorstDeveloper['year'].max()
    año_min = df_funcion_UsersWorstDeveloper['year'].min()

    if año < año_min or año > año_maximo:
        
        print('No hay datos para el año indicado!')
        print('Se debe colocar un año entre', año_min, 'y', año_maximo)
    
    else:
        # Filtro el año dado con recomendación igual a 'False' y sentimiento negativo
        filtro_reviews = df_funcion_UsersWorstDeveloper[
            (df_funcion_UsersWorstDeveloper['year'] == año) &
            (df_funcion_UsersWorstDeveloper['recommend'] == False) &
            (df_funcion_UsersWorstDeveloper['sentiment_analysis'] == 0)]

        # Agrupo por item_id y cuento la cantidad de comentarios negativos
        sentiment_count = filtro_reviews.groupby('developer')['sentiment_analysis'].count().reset_index(name='sentiment_count')

        # Ordeno de forma descendente el conteo de 'sentiment_count'
        ordenados_sentiments = sentiment_count.sort_values(by='sentiment_count', ascending=False)

        # Elimino las filas con NaN
        # df_merge = df_merge.dropna()

        # Filtro el top 3
        top_3_worst_developer = ordenados_sentiments.head(3)

        # Crea la lista de resultados en el formato deseado con nombres de juegos
        result = [{"Puesto {}: ".format(i+1): item_name} for i, item_name in enumerate(top_3_worst_developer['developer'])]

        

        return result

In [112]:
# Ejemplo de uso
año_deseado = 2013
resultado = UsersWorstDeveloper(año_deseado)
resultado

[{'Puesto 1: ': 'Valve'},
 {'Puesto 2: ': 'Terry Cavanagh'},
 {'Puesto 3: ': 'Facepunch Studios'}]

Comentario: <br>
Hay muchos NaN, que no contienen los datos del juego y developer, por lo tanto en algunos años <br>
no terminan de juntar datos!

In [113]:
# Guardo los DataFrame utilizados en un .csv
# df_funcion_UsersWorstDeveloper.to_csv("df_funciones/df_funcion_UsersWorstDeveloper.csv", index=False)


# Comentado, por que se hace una sola vez!

-------
-----
-----

# FUNCIÓN 5

---
---
---

# Función: Sentiment_analysis

def sentiment_analysis( empresa desarrolladora : str ): Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.<br>

Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}

In [114]:
steam_games_fc5 = steam_games.copy()
users_reviews_fc5 = users_reviews.copy()
users_items_fc5 = users_items.copy()


In [115]:
# Selecciono las columnas deseadas
col_sg_5 = ['title','developer']
col_ur_5 = ['item_id', 'sentiment_analysis']
col_ui_5 = ['item_id', 'item_name']

# Genero los dataframe con esas columnas
steam_games_fc5 = steam_games_fc5[col_sg_5]
users_reviews_fc5 = users_reviews_fc5[col_ur_5]
users_items_fc5 = users_items_fc5[col_ui_5]

In [116]:
# Unir los DataFrames usando 'title' y 'item_name'
df_name_developer = pd.merge(users_items_fc5, steam_games_fc5, left_on='item_name', right_on='title', how='left')

# Eliminar la columna 'title' duplicada
df_name_developer = df_name_developer.drop(columns='title')

# Elimino los campos con NaN
df_name_developer.dropna(inplace=True)

# Elimino los duplicados
df_name_developer.drop_duplicates(inplace=True)

# Reseteo los indices
df_name_developer = df_name_developer.reset_index(drop=True)

In [117]:
df_name_developer

,item_id,item_name,developer
0,10.0,Counter-Strike,Valve
1,20.0,Team Fortress Classic,Valve
2,30.0,Day of Defeat,Valve
3,40.0,Deathmatch Classic,Valve
4,50.0,Half-Life: Opposing Force,Gearbox Software
...,...,...,...
5832,381890.0,Induction,Bryan Gale
5833,443700.0,The Inner Sea,dsquare
5834,433400.0,Kim,The Secret Games Company
5835,444770.0,Mimpi Dreams,Silicon Jelly


In [118]:
# Unir los DataFrames usando 'item_id' para tener la columna de 'sentiment_analysis'
df_funcion_sentiment_analysis = pd.merge(df_name_developer, users_reviews_fc5, on='item_id', how='left')

In [119]:
df_funcion_sentiment_analysis

,item_id,item_name,developer,sentiment_analysis
0,10.0,Counter-Strike,Valve,2.0
1,10.0,Counter-Strike,Valve,1.0
2,10.0,Counter-Strike,Valve,2.0
3,10.0,Counter-Strike,Valve,2.0
4,10.0,Counter-Strike,Valve,2.0
...,...,...,...,...
39568,381890.0,Induction,Bryan Gale,NaN
39569,443700.0,The Inner Sea,dsquare,NaN
39570,433400.0,Kim,The Secret Games Company,NaN
39571,444770.0,Mimpi Dreams,Silicon Jelly,NaN


In [120]:
@app.get('/sentiment_analysis/{developer}')
def sentiment_analysis(developer: str):
    # Genero un filtro por developer que entra como input
    df_filtro_fc5 = df_funcion_sentiment_analysis[(df_funcion_sentiment_analysis['developer'] == developer)]

    # Contar las filas donde sentiment_analysis es igual a 0, generando un booleano y luego suma ese True
    valor_0 = df_filtro_fc5['sentiment_analysis'].eq(0).sum()
    valor_1 = df_filtro_fc5['sentiment_analysis'].eq(1).sum()
    valor_2 = df_filtro_fc5['sentiment_analysis'].eq(2).sum()
    # valor_Nan = df_filtro_fc5['sentiment_analysis'].isna().sum()

    # Crear un diccionario para la respuesta
    resultados_dict = {developer: {'Positive': valor_2, 'Neutral': valor_1, 'Negative': valor_0 }}

    return resultados_dict

In [121]:
# Ejemplo de uso
nombre_desarrolladora = 'Ion Storm'
resultado = sentiment_analysis(nombre_desarrolladora)
print(resultado)

{'Ion Storm': {'Positive': 13, 'Neutral': 6, 'Negative': 3}}


In [122]:
# filtro de developers para probar funcion
valores_unicos = df_funcion_sentiment_analysis['developer'].unique()
valores_unicos_df = pd.DataFrame(valores_unicos)
valores_unicos_df

,0
0,Valve
1,Gearbox Software
2,Double Fine Productions
3,Firaxis Games
4,Ion Storm
...,...
4196,Sheado.net
4197,Bryan Gale
4198,dsquare
4199,Silicon Jelly


In [123]:
# Ejemplo de uso
nombre_desarrolladora = 'Firaxis Games'
resultado = sentiment_analysis(nombre_desarrolladora)
print(resultado)

{'Firaxis Games': {'Positive': 2, 'Neutral': 0, 'Negative': 3}}


In [124]:
# Guardo los DataFrame utilizados en un .csv
# df_funcion_sentiment_analysis.to_csv("df_funciones/df_funcion_sentiment_analysis.csv", index=False)


# Comentado, por que se hace una sola vez!

Solucion al problema que genera la función 5 en la API pero corre bien en el ipynb <br>
Se lo paso a tipo de dato int

In [125]:
@app.get('/sentiment_analysis/{developer}')
def sentiment_analysis(developer: str):
    # Genero un filtro por developer que entra como input
    df_filtro_fc5 = df_funcion_sentiment_analysis[(df_funcion_sentiment_analysis['developer'] == developer)]

    # Contar las filas donde sentiment_analysis es igual a 0, generando un booleano y luego suma ese True
    valor_0 = int(df_filtro_fc5['sentiment_analysis'].eq(0).sum())
    valor_1 = int(df_filtro_fc5['sentiment_analysis'].eq(1).sum())
    valor_2 = int(df_filtro_fc5['sentiment_analysis'].eq(2).sum())

    # Crea el diccionario para la respuesta
    resultados_dict = {developer: {'Positive': valor_2, 'Neutral': valor_1, 'Negative': valor_0}}

    return resultados_dict
